In [2]:
# 助教把所有你可能會用到的套件都先裝進來了，你可以再看情況自行變動
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

print("套件導入完成！")

套件導入完成！


## 讀取資料

In [3]:
# 讀取訓練集和測試集
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(f"訓練集: {len(train)} 筆")
print(f"測試集: {len(test)} 筆")
print(f"\n訓練集前 3 筆:")
print(train.head(3))

訓練集: 600 筆
測試集: 200 筆

訓練集前 3 筆:
   Age Sex      BP Cholesterol  Na_to_K   Drug
0   36   M     LOW        HIGH   26.332  DrugY
1   68   M  NORMAL      NORMAL   21.684  DrugY
2   57   F     LOW      NORMAL   21.185  drugX


## 準備資料

In [4]:
# 定義映射字典 (將文字轉為數字)
# 這裡我們手動定義，以保留 BP 的順序性 (Low < Normal < High)
sex_map = {'F': 0, 'M': 1}
bp_map = {'LOW': 0, 'NORMAL': 1, 'HIGH': 2}
chol_map = {'NORMAL': 0, 'HIGH': 1}

# 複製一份資料以免改到原始檔
train_processed = train.copy()
test_processed = test.copy()

# 對訓練集進行轉換
train_processed['Sex'] = train_processed['Sex'].map(sex_map)
train_processed['BP'] = train_processed['BP'].map(bp_map)
train_processed['Cholesterol'] = train_processed['Cholesterol'].map(chol_map)

# 對測試集進行轉換 (必須做一樣的處理)
test_processed['Sex'] = test_processed['Sex'].map(sex_map)
test_processed['BP'] = test_processed['BP'].map(bp_map)
test_processed['Cholesterol'] = test_processed['Cholesterol'].map(chol_map)

# 選取所有特徵
features = ['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']

# 用於訓練的特徵
X_train = train_processed[features]
y_train = train_processed['Drug']

# 用來進行測試的特徵
X_test = test_processed[features]

print(f"訓練特徵: {X_train.shape}")
print(f"訓練目標: {y_train.shape}")
print(f"測試特徵: {X_test.shape}")
print("前幾筆處理後的特徵資料：")
print(X_train.head())

訓練特徵: (600, 5)
訓練目標: (600,)
測試特徵: (200, 5)
前幾筆處理後的特徵資料：
   Age  Sex  BP  Cholesterol  Na_to_K
0   36    1   0            1   26.332
1   68    1   1            0   21.684
2   57    0   0            0   21.185
3   43    0   2            0   24.613
4   28    1   0            1   21.557


## 模型訓練

In [5]:
# 使用隨機森林分類器，參數設定 random_state 固定結果
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# HINT: 訓練模型 
model.fit(X_train, y_train)

# 看看訓練集上的準確率
train_score = model.score(X_train, y_train) # 括弧內部的內容要填，才能看到訓練集上的準確率
print(f"訓練集準確率: {train_score:.2%}")
print("\n✅ 模型訓練完成！")

訓練集準確率: 87.00%

✅ 模型訓練完成！


## 用訓練好的模型直接進行預測

In [6]:
# 直接預測測試集
predictions = model.predict(X_test) # 括弧內部的內容要填，才能進行預測，並做資料繳交

print(f"✅ 預測完成！共 {len(predictions)} 筆")
print(f"\n預測結果分布:")
print(pd.Series(predictions).value_counts())

✅ 預測完成！共 200 筆

預測結果分布:
DrugY    133
drugX     45
drugC     14
drugA      5
drugB      3
Name: count, dtype: int64


## 資料繳交

In [7]:
# 創建提交檔案
submission = pd.DataFrame({
    'Id': range(len(predictions)),
    'Drug': predictions
})

# 儲存
submission.to_csv('submission.csv', index=False)

print("✅ 提交檔案已創建: submission.csv")
print(f"\n前 10 筆預測:")
print(submission.head(10))

✅ 提交檔案已創建: submission.csv

前 10 筆預測:
   Id   Drug
0   0  DrugY
1   1  DrugY
2   2  DrugY
3   3  DrugY
4   4  drugX
5   5  drugX
6   6  DrugY
7   7  DrugY
8   8  DrugY
9   9  DrugY


# 報告

姓名：林宥臣 學號：111303547

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：0.79000  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：加入了所有特徵 (Age, Sex, BP, Cholesterol)。針對類別資料進行編碼處理：將 Sex 轉為 0/1，將 BP 依據程度 (LOW/NORMAL/HIGH) 映射為 0/1/2，Cholesterol 映射為 0/1。模型更換為 RandomForestClassifier (隨機森林)。  
    結果與觀察 (分數變化、心得等)：隨機森林能夠很好地捕捉特徵之間的非線性關係（例如：年齡大且血壓高的人適合某種藥），且將所有資訊數位化後，模型有了充分的判斷依據。  
    該次實驗分數： 0.79000  
【實驗 2】  
    我做的修改：未做實驗2  
    結果與觀察 (分數變化、心得等)：未做實驗2  
    該次實驗分數：未做實驗2  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：
    (1) 最有效的修改： 我認為最有效的修改是 「特徵處理（Data Preprocessing）」。這個作業再次證明，模型再強，如果沒有給它正確的資料（例如將 BP、Cholesterol 文字轉數值），它也無法運作。把所有特徵都納入訓練是成功的關鍵。
    (2) 學習與啟發： 這次實驗讓我學到，在醫療資料中，特徵往往混合了數值（如年齡、指數）與類別（如高/低、男/女）。學會如何將這些文字標籤轉換為模型讀得懂的數字（Mapping），是機器學習最基礎也最重要的技能。

